In [62]:
from llama_index.llms import Bedrock, ChatMessage
from databricks.sdk import WorkspaceClient


messages = [
    ChatMessage(role="user", content="Tell me a story"),
]

llm = Bedrock(model="anthropic.claude-instant-v1",
              additional_kwargs={"max_tokens_to_sample": 8000})
output = llm.chat(messages)
print(output)

assistant:  Here is a short story for you:

Once upon a time, in a land far away, there lived a little girl named Lucy. Lucy loved to explore the forest near her village. One sunny afternoon, as Lucy was walking through the trees, she came upon a small clearing with a beautiful flowing stream. 

Lucy walked over to the stream to get a closer look. That's when she noticed something strange - there was a tiny fairy trapped in the current! The fairy's wing seemed to be hurt and she couldn't fly out of the water. 

"Help!" cried the fairy. "Please help me!"

Lucy didn't hesitate. She carefully waded into the stream and scooped the fairy up in her hands. The fairy was so small, she fit in Lucy's palm. 

"Thank you for saving me," said the fairy. "My name is Fiona. How can I repay your kindness?"

Lucy smiled. "You don't owe me anything. I'm just glad I could help."

Just then, Fiona had an idea. "I know! Come with me and I'll show you a magic place in the forest that no human has ever seen 

In [16]:
from databricks.connect import DatabricksSession
import pandas as pd

# Create the Databricks session
spark = DatabricksSession.builder.getOrCreate()

# Read the samples catalog, nyctaxi schema, and trips table
df = spark.read.table("samples.nyctaxi.trips")

In [17]:
# Read local file into a pandas dataframe
pdf = pd.read_csv('data/dataset_best-linkedin-jobs-scraper_2023-10-25_12-15-39-346.csv')

In [69]:
# Convert the pandas dataframe to a spark dataframe
df1 = spark.createDataFrame(pdf)
#df1.describe().show()

# Write the spark dataframe to a table in the Databricks cluster
df1.write.saveAsTable("research_and_development.indeed_jobs.linkedin_jobs")

+-------+------------+----------------+----------------+----------------+----------------+----------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|summary|     company|criteria/0/title|criteria/0/value|criteria/1/title|criteria/1/value|criteria/2/title|    criteria/2/value|criteria/3/title|    criteria/3/value|          datePosted|         description|employmentType|                  id|            industry|                link|            location|               title|
+-------+------------+----------------+----------------+----------------+----------------+----------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  count|    

In [95]:
# Write the spark dataframe to a csv file in the catalog and schema volume
df1.coalesce(1).write.format("csv").option("header", "true").save("/Volumes/research_and_development/indeed_jobs/vol1/linkedin_jobs", mode="overwrite")

In [23]:
df1.show()

+--------------------+----------------+----------------+----------------+----------------+----------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+
|             company|criteria/0/title|criteria/0/value|criteria/1/title|criteria/1/value|criteria/2/title|    criteria/2/value|criteria/3/title|    criteria/3/value|          datePosted|         description|employmentType|        id|            industry|                link|            location|               title|
+--------------------+----------------+----------------+----------------+----------------+----------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+
|                 ICF| Seniority level|    

In [27]:
df1.dtypes

[('company', 'string'),
 ('criteria/0/title', 'string'),
 ('criteria/0/value', 'string'),
 ('criteria/1/title', 'string'),
 ('criteria/1/value', 'string'),
 ('criteria/2/title', 'string'),
 ('criteria/2/value', 'string'),
 ('criteria/3/title', 'string'),
 ('criteria/3/value', 'string'),
 ('datePosted', 'string'),
 ('description', 'string'),
 ('employmentType', 'string'),
 ('id', 'bigint'),
 ('industry', 'string'),
 ('link', 'string'),
 ('location', 'string'),
 ('title', 'string')]

In [28]:
df1.select("title").show()

+--------------------+
|               title|
+--------------------+
|Data Engineer, Az...|
|Security Software...|
|Software Engineer...|
|ESG Jr. Data Mana...|
|ServiceNow Softwa...|
|ServiceNow Develo...|
+--------------------+



In [42]:
rows = df1.take(5)

In [49]:
keys = rows[0].asDict().keys()

In [57]:
from llama_index import Document
docs = []
for row in rows:
    curr_doc = Document()
    metadata = {}
    for key in keys:
        if key == 'description':
            curr_doc.text = row[key]
        else:
            metadata[key] = row[key]
    curr_doc.metadata = metadata
    docs.append(curr_doc)

In [59]:
print(len(docs))

5


In [66]:
from llama_index.query_engine import PandasQueryEngine
from llama_index import ServiceContext
from llama_index.embeddings import HuggingFaceEmbedding
panda = df1.toPandas()
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

query_engine = PandasQueryEngine(panda, verbose=True, service_context=service_context)
response = query_engine.query("Which job uses pyspark")
response


> Pandas Instructions:
```
 df[df['description'].str.contains('pyspark')]
```
> Pandas Output: There was an error running the output as Python code. Error message: unexpected indent (<unknown>, line 1)


Traceback (most recent call last):
  File "/home/bphil/.local/lib/python3.10/site-packages/llama_index/query_engine/pandas_query_engine.py", line 59, in default_output_processor
    tree = ast.parse(output)
  File "/usr/lib/python3.10/ast.py", line 50, in parse
    return compile(source, filename, mode, flags,
  File "<unknown>", line 1
    df[df['description'].str.contains('pyspark')]
IndentationError: unexpected indent


Response(response='There was an error running the output as Python code. Error message: unexpected indent (<unknown>, line 1)', source_nodes=[], metadata={'pandas_instruction_str': " df[df['description'].str.contains('pyspark')]"})

In [67]:
panda[panda['description'].str.contains('pyspark')]

,company,criteria/0/title,criteria/0/value,criteria/1/title,criteria/1/value,criteria/2/title,criteria/2/value,criteria/3/title,criteria/3/value,datePosted,description,employmentType,id,industry,link,location,title


In [ ]:
csv_sparf = spark.read.csv('/Volumes/research_and_development/indeed_jobs/vol1/linkedin_jobs', header=True,multiLine=True)
csv_sparf.show()
todf = csv_sparf.toPandas()
todf

+--------------------+----------------+----------------+----------------+----------------+----------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+
|             company|criteria/0/title|criteria/0/value|criteria/1/title|criteria/1/value|criteria/2/title|    criteria/2/value|criteria/3/title|    criteria/3/value|          datePosted|         description|employmentType|        id|            industry|                link|            location|               title|
+--------------------+----------------+----------------+----------------+----------------+----------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+
|                 ICF| Seniority level|    

,company,criteria/0/title,criteria/0/value,criteria/1/title,criteria/1/value,criteria/2/title,criteria/2/value,criteria/3/title,criteria/3/value,datePosted,description,employmentType,id,industry,link,location,title
0,ICF,Seniority level,Entry level,Employment type,Full-time,Job function,Information Technology,Industries,Business Consulting and Services,2023-10-25T04:37:27.000Z,* We are open to supporting 100% remote work a...,FULL_TIME,3677646750,Business Consulting and Services,https://www.linkedin.com/jobs/view/data-engine...,"Reston, VA","Data Engineer, Azure-based Health Data System ..."
1,Paylocity,Seniority level,Entry level,Employment type,Full-time,Job function,Engineering and Information Technology,Industries,Software Development,None,Job Type\n\nFull-time\n\nDescription\n\nWhen y...,None,3731724506,None,https://www.linkedin.com/jobs/view/security-so...,United States,Security Software Engineer
2,"CARIAD, Inc.",Seniority level,Entry level,Employment type,Full-time,Job function,Engineering and Information Technology,Industries,Software Development,2023-10-25T05:11:37.000Z,"We are CARIAD , an automotive software devel...",FULL_TIME,3671215760,Software Development,https://www.linkedin.com/jobs/view/software-en...,"Santa Clara, CA",Software Engineer - HW Platform Development
3,Montrose Environmental Group,Seniority level,Entry level,Employment type,Full-time,Job function,Information Technology,Industries,Environmental Services,2023-10-25T09:31:30.000Z,"About You\nAre YOU enthusiastic, self-motivate...",FULL_TIME,3703412950,Environmental Services,https://www.linkedin.com/jobs/view/esg-jr-data...,"Texas, United States",ESG Jr. Data Management Consultant - Computer ...
4,SAIC,Seniority level,Entry level,Employment type,Full-time,Job function,Engineering and Information Technology,Industries,"IT Services and IT Consulting, Defense and Spa...",2023-10-25T10:14:33.000Z,"Job ID 2312607\n\nLocation REMOTE WORK, VA, US...",FULL_TIME,3719329758,"IT Services and IT Consulting,Defense and Spac...",https://www.linkedin.com/jobs/view/servicenow-...,"Virginia, United States",ServiceNow Software Engineer - VIRTUAL
5,SAIC,Seniority level,Entry level,Employment type,Full-time,Job function,Engineering and Information Technology,Industries,"Computer Hardware Manufacturing, Defense and S...",2023-10-25T09:07:15.000Z,"Job ID 2312613\n\nLocation REMOTE WORK, VA, US...",FULL_TIME,3719328820,"Computer Hardware Manufacturing,Defense and Sp...",https://www.linkedin.com/jobs/view/servicenow-...,"Virginia, United States","ServiceNow Developer, Principal - VIRTUAL"
